In [1]:
import emip_toolkit as emtk
import filter_fixation as ff
import numpy as np
import pandas as pd
import json

%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%load_ext autoreload
%autoreload 2

In [11]:
def draw(centers):
    fixs = []
    for data in centers.values():
        if data == []: continue
        else:
            x, y = data
            fixs.append([x, y])
    fixs = np.array(fixs)
    l = plt.plot(fixs[:, 0], fixs[:, 1], alpha=0.4, c='orange')
    line = l.pop(0)
    scatter = plt.scatter(fixs[:, 0], fixs[:, 1], alpha=0.5, s=100, c='yellow')
    return line, scatter

def is_fixation(x, y, centers):
    if find_index(centers):
        return None
    for index, (c_x, c_y) in centers.items():
        # If click is on one fixation -> Hide it
        if c_x-12 < x < c_x+12 and c_y-12 < y < c_y+12:
            return index
    return None

def find_index(centers):
    for index, data in centers.items():
        if data == []:
            return index
    return None

In [33]:
fig = plt.figure(figsize=(12,8))

path = 'datasets/GazeBase_TEX/Data/S_2011_S1_TEX.csv'

pixel_data = ff.process_GazeBase(path)
fixations, saccades = ff.filter_fixations(pixel_data)

path = path.split('/')[-1]
trial_id = int(path[8])
round_id = path[2]
participant_id = path[3:6]
imagepath = f"datasets/GazeBase_TEX/StimuliExamples/TEXScreenshots/TEX_R{round_id}S{trial_id}_bg.png"

img = mpimg.imread(imagepath)

# Initialize fixation centers
centers = {}
for index, fixation in fixations.items():
    fixation = fixation.get_fixation()
    x = fixation[4]
    y = fixation[5]
    centers[index] = [x,y]

line, scatter = draw(centers)

def onclick(event):
    x, y = event.xdata, event.ydata
    global line, scatter, centers
    index = is_fixation(x, y, centers)

    if index:
        # If click is on one fixation -> Hide it
        centers[index] = []
    else:
        # If click is on empty spot -> Find empty spot and fill with current position
        index = find_index(centers)
        if index:
            centers[index] = [x, y]

    line.remove()
    scatter.remove()
    
    line, scatter = draw(centers)
    fig.canvas.draw()
    
    with open(f'corrections/R{round_id}S{trial_id}_{participant_id}.txt', 'w') as file:
        file.write(json.dumps(centers))

cid = fig.canvas.mpl_connect('button_press_event', onclick)
imgplot = plt.imshow(img)
plt.show()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/tkinter/__init__.py", line 1883, in __call__
    return self.func(*args)
  File "/Users/speng/Library/Python/3.8/lib/python/site-packages/matplotlib/backends/_backend_tk.py", line 293, in button_press_event
    FigureCanvasBase.button_press_event(
  File "/Users/speng/Library/Python/3.8/lib/python/site-packages/matplotlib/backend_bases.py", line 1854, in button_press_event
    self.callbacks.process(s, mouseevent)
  File "/Users/speng/Library/Python/3.8/lib/python/site-packages/matplotlib/cbook/__init__.py", line 229, in process
    self.exception_handler(exc)
  File "/Users/speng/Library/Python/3.8/lib/python/site-packages/matplotlib/cbook/__init__.py", line 81, in _exception_printer
    raise exc
  File "/Users/speng/Library/Python/3.8/lib/python/site-packages/matplotlib/cbook/__init__.py", line 224, in pro